<h1 style='text-align: center;'>DATA RETRIEVAL</h1>


<h3 style='text-align: center;'>DATA RETRIEVAL FLOW CHART</h3>


<img src="../assets/FLOWCHART.png">

### Import the libraries


In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

from collections import defaultdict
import os
from tqdm import tqdm

from dotenv import dotenv_values

### Data Extraction

In [2]:
def data_retrieval(path):
    """
        extracts the data in specific rows and columns in different excel files.
    
        Parameters
        ------------
        path: str
            the path to the folder in which contains all the subfolders of excel files

        Returns
        ------------
        None
    
    """
    # DECLARE GLOBAL VARIABLE
    global df
    
    # ACCESS THE SHEET `Urinalysis` INSIDE THE EXCEL FILE
    EXCEL_FILE = pd.read_excel(path, sheet_name = "Urinalysis")
    
    CONTAINER = defaultdict(lambda: np.nan)

    # EXTRACT THE PATIENT'S AGE
    PATIENT_AGE = EXCEL_FILE.iloc[7, 2]
    
    # TEST IF THE CELL CONTAINING THE AGE IS BLANK OR NULL
    if not pd.isna(PATIENT_AGE):
        if PATIENT_AGE.endswith("YEARS OLD") or PATIENT_AGE.endswith("YEAR OLD"):
            CONTAINER["Age"] = int("".join(char for char in PATIENT_AGE if char.isnumeric()))
        else:
            CONTAINER["Age"] = int("".join(char for char in PATIENT_AGE if char.isnumeric()))/10
    else:
        return f"{path} contains empty cell"
    
    # EXTRACT THE PATIENT'S GENDER
    PATIENT_GENDER = EXCEL_FILE.iloc[7, 4]
    CONTAINER["Gender"] = PATIENT_GENDER
    
    INDECIES_AND_FEATURE_NAMES = ((17, "Color"),
                                  (18, "Transparency"),
                                  (21, "Glucose"),
                                  (22, "Protein"),
                                  (23, "pH"),
                                  (24, "Specific Gravity"),
                                  (27, "WBC"),
                                  (28, "RBC"),
                                  (29, "Epithelial Cells"),
                                  (30, "Mucous Threads"),
                                  (31, "Amorphous Urates"),
                                  (32, "Bacteria"))
    
    COLUMN_NUMBER = 5
    
    # EXTRACT THE REMAINING DATA INSIDE THE EXCEL FILE; SINCE THESE DATA HAVE THE SAME COLUMN POSITION, ONLY THE ROW IS MODIFIED ALONG THE ITERATION
    for row, column_name in INDECIES_AND_FEATURE_NAMES:
        CONTAINER[column_name] = EXCEL_FILE.loc[row][COLUMN_NUMBER]

    # UPDATE THE GLOBAL VARIABLE
    df = df._append(CONTAINER, ignore_index=True)

In [3]:
# ACCESS THE FOLDER DIRECTORY
DIRECTORY_PATH = dotenv_values(".env.local")["DATASET_PATH"]
DIRECTORY_FOLDERS = os.listdir(DIRECTORY_PATH)

# INITIALIZE THE DATA FRAME
df = pd.DataFrame(columns = ["Age", "Gender", 
                             "Color", "Transparency",
                             "Glucose", "Protein", "pH", "Specific Gravity", 
                             "WBC", "RBC", "Epithelial Cells", "Mucous Threads", "Amorphous Urates", "Bacteria"])

# ITERATE THROUGH EACH FOLDER IN THE DIRECTORY PATH AND EXTRACT THE PATH OF EACH EXCEL FILE INSIDE EACH FOLDER INSIDE THE DIRECTORY PATH
for subfolder_name in tqdm(DIRECTORY_FOLDERS, position=0, leave=False):
    SUBFOLDER_PATH = os.path.join(DIRECTORY_PATH, subfolder_name)
    
    EXCEL_FILES = os.listdir(SUBFOLDER_PATH)

    for excel_name in EXCEL_FILES:
        PATH_ = rf"{SUBFOLDER_PATH}\{excel_name}"
        data_retrieval(path=PATH_)

  0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
FILE_NAME = "../data/UTI DATASET-UNLABELED.xlsx"
df.to_excel(FILE_NAME, index=False)